# **Анализ результатов A/B-тестирования: кумулятивные метрики**

Итак, давайте рассмотрим, как проводится анализ результатов A/B-тестирования (в нашем алгоритме он был обозначен под номером 6) с помощью Python.

![](https://lms.skillfactory.ru/assets/courseware/v1/ba083e24b47b02b96efcd0fa4566fa5e/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst-eda-5-9.png)

В качестве примера давайте попробуем решить задачу об анализе конверсии на некотором интернет-магазине, который занимается продажей различных товаров. 

***
### **ОПИСАНИЕ ЗАДАЧИ**

Проводится A/B-тестирование для двух вариантов рекомендаций сопутствующих товаров в профиле товара некоторого интернет-магазина. 

* Вариант 1. Дополнительные товары рекомендуются на основе статистических данных уже совершённых покупок — товары, которые другие пользователи часто покупают с основным. 
* Вариант 2. Дополнительные товары рекомендуются на основе совершённых пользователем ранее покупок с помощью ML-модели.

Варианты сравниваются по доле пользователей, которые добавляют хотя бы один из предложенных товаров в корзину (метрикой является конверсия действия **«добавить в корзину сопутствующий товар**»).

Для этого пользователи были разделены на две группы. 

Контрольной (группе А) доступен первый вариант рекомендаций.
Тестовой (группе B) доступен второй вариант. 
Предполагается, что все условия и принципы проведения A/B-теста были соблюдены, то есть:

✔️ выборка имеет достаточный для принятия решения объём;

✔️ выбран достаточный интервал времени для тестирования;

✔️ группы максимально сбалансированы;

✔️ внешнее вмешательство минимально.

Итак, в результате проведения A/B-тестирования были получены следующие данные:

[данные](https://lms.skillfactory.ru/assets/courseware/v1/f9f05ba6ecb36996a8ec5c6e89cd2a75/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/ab_data.zip)



***
### **ЗНАКОМИМСЯ С ДАННЫМИ**


In [2]:
# импортируем библиотеки, которые нам пригодятся:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
ab_data = pd.read_csv('data/ab_data.csv')
display(ab_data.head())
display(ab_data.info())

,user_id,timestamp,group,converted
0,851104,2017-01-21,A,0
1,804228,2017-01-12,A,0
2,661590,2017-01-11,B,0
3,853541,2017-01-08,B,0
4,864975,2017-01-21,A,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290482 entries, 0 to 290481
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    290482 non-null  int64 
 1   timestamp  290482 non-null  object
 2   group      290482 non-null  object
 3   converted  290482 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 8.9+ MB


None

In [9]:
# нам нужно перевести столбец с информацией о дате события (timestamp) в формат datetime
ab_data['timestamp'] = pd.to_datetime(ab_data['timestamp'], format='%Y-%m-%d')

В итоге мы получили таблицу, в которой отображены входы клиента на сайт (сессия) и его действия. Для того, чтобы получить показатель конверсии в группах на каждый день, нам необходимо предварительно сгруппировать наши данные по дате и группе и вычислить количество пользователей, заходивших на сайт, и суммарное количество совершённых целевых действий. 

Дополнительно для удобства сбросим индекс на сгруппированной таблице c помощью метода **reset_index()** и переименуем столбец user_id в users_count с помощью метода **rename()**.

In [12]:
daily_data = ab_data.groupby(['timestamp','group']).agg({
    'user_id':'count',
    'converted':'sum'
}).reset_index().rename(columns={'user_id': 'users_count'})
daily_data.head()

,timestamp,group,users_count,converted
0,2017-01-02,A,2851,357
1,2017-01-02,B,2848,339
2,2017-01-03,A,6584,749
3,2017-01-03,B,6620,754
4,2017-01-04,A,6573,797


Примечание. Количество пользователей в день в контрольной и тестовой группе определяется как количество значений в столбце user_id в сгруппированных данных, а количество совершённых целевых действий — это просто сумма по столбцу converted (1 в строке означает положительный исход, то есть целевое действие было совершено, а 0 — отрицательный исход; просуммировав все 1 и 0 в группе, получим количество положительных исходов).


In [27]:
# Рассчитайте конверсию на каждый день
# Результат занесите в столбец conversion. Чему была равна конверсия (в процентах)
# в группе А 5 января 2017 года? Ответ округлите до второго знака после запятой.

daily_data['conversion'] = daily_data['converted']/daily_data['users_count']*100
display(daily_data.head())
display(daily_data[(daily_data['timestamp'] == '2017-01-05')&(daily_data['group'] == 'A')]['conversion'].round(2))

,timestamp,group,users_count,converted,conversion
0,2017-01-02,A,2851,357,12.521922
1,2017-01-02,B,2848,339,11.903090
2,2017-01-03,A,6584,749,11.376063
3,2017-01-03,B,6620,754,11.389728
4,2017-01-04,A,6573,797,12.125361


6    12.32
Name: conversion, dtype: float64

***
### **ВИЗУАЛЬНЫЙ АНАЛИЗ КОНВЕРСИИ В ГРУППАХ**
